In [12]:
import os

def download_subway_video():
    url = "https://www.youtube.com/watch?v=RbVMiu4ubT0"
    
    command = (
        'yt-dlp '
        '-f "bv*+ba/b" '                          # Best video+audio combined (fallback: best)
        '--merge-output-format mp4 '              # Merge into MP4 if separate streams
        '--user-agent "Mozilla/5.0" '             # Bypass some 403 errors
        '--no-playlist '                          # Avoid playlist downloading
        '-o "subway.mp4" '                        # Output filename
        f'"{url}"'
    )
    
    os.system(command)


In [13]:
download_subway_video()

[youtube] Extracting URL: https://www.youtube.com/watch?v=RbVMiu4ubT0
[youtube] RbVMiu4ubT0: Downloading webpage
[youtube] RbVMiu4ubT0: Downloading tv client config
[youtube] RbVMiu4ubT0: Downloading tv player API JSON
[youtube] RbVMiu4ubT0: Downloading ios player API JSON
[youtube] RbVMiu4ubT0: Downloading m3u8 information
[info] RbVMiu4ubT0: Downloading 1 format(s): 303+251
[download] Destination: subway.f303.webm
[download] 100% of   73.48MiB in 00:00:04 at 17.04MiB/s    
[download] Destination: subway.f251.webm
[download] 100% of   64.54KiB in 00:00:00 at 688.49KiB/s   
[Merger] Merging formats into "subway.mp4"
Deleting original file subway.f251.webm (pass -k to keep)
Deleting original file subway.f303.webm (pass -k to keep)


In [14]:
import subprocess
import imageio_ffmpeg

def burn_audio_and_styled_subtitles(
    video_path="subway.mp4",
    audio_path="output.wav",
    subtitle_path="output.ass",  # Use .ass for styling
    output_path="output_final.mp4"
):
    ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()

    command = [
        ffmpeg_path,
        "-i", video_path,                 # Input video
        "-i", audio_path,                 # Input audio
        "-vf", f"ass={subtitle_path}",   # Burn styled subtitles
        "-c:a", "aac",                    # Audio: encode to AAC
        "-c:v", "libx264",                # Video: encode to x264 (needed for hardsubs)
        "-shortest",                      # Trim to shortest input
        output_path
    ]

    try:
        subprocess.run(command, check=True)
        print(f"✅ Final video saved to: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"❌ FFmpeg error:\n{e}")

burn_audio_and_styled_subtitles()


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2.5)
  configuration: --prefix=/Volumes/tempdisk/sw --extra-cflags=-fno-stack-check --arch=arm64 --cc=/usr/bin/clang --enable-gpl --enable-libvmaf --enable-libopenjpeg --enable-libopus --enable-libmp3lame --enable-libx264 --enable-libx265 --enable-libvpx --enable-libwebp --enable-libass --enable-libfreetype --enable-fontconfig --enable-libtheora --enable-libvorbis --enable-libsnappy --enable-libaom --enable-libvidstab --enable-libzimg --enable-libsvtav1 --enable-libharfbuzz --enable-libkvazaar --pkg-config-flags=--static --enable-ffplay --enable-postproc --enable-neon --enable-runtime-cpudetect --disable-indev=qtkit --disable-indev=x11grab_xcb
  libavutil      59. 39.100 / 59. 39.100
  libavcodec     61. 19.100 / 61. 19.100
  libavformat    61.  7.100 / 61.  7.100
  libavdevice    61.  3.100 / 61.  3.100
  libavfilter    10.  4.100 / 10.  4.100
  libswscale      8. 

KeyboardInterrupt: 